# Exploratory Data Analysis

So until now, we have not idea whether all our efforts are gonna be invain or not; But by doing the analysis here, we will know with high confidence whether the data is useful or not.

We do this by analysing the followings:
1. **Most common words** - find these and create word clouds
2. **Size of vocabulary** - look number of unique words and also how quickly someone speaks
3. **Amount of profanity** - most common terms

## Load the cleaned data

In [14]:
import pandas as pd

data = pd.read_csv('saves/cleaned_transcripts_df.csv', index_col = 0)
data

,Transcript
Lousic C.K.,intro fade the music out let 's roll hold ther...
Dave Chappelle,this be dave he tell dirty joke for a living t...
Ricky Gervais,hello hello how you do great thank you wow cal...
Bo Burham,bo what old macdonald have a farm e I e I o an...
Bill Burr,all right thank you thank you very much thank ...
Jim Jefferies,lady and gentleman please welcome to the stage...
John Mulaney,armed with boyish charm and a sharp wit the fo...
Hasan Minhaj,what be up davis what be up I be home I have t...
Ali Wong,lady and gentleman please welcome to the stage...
Anthony Jeselnik,thank you thank you thank you san francisco th...


## Vectorize the data

In [40]:
# Imports
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Vectorize
vectorizer = CountVectorizer(stop_words = 'english')
vectorized_data = vectorizer.fit_transform(data['Transcript'])

# Convert to DataFrame
vectorized_df = pd.DataFrame(vectorized_data.toarray(), columns = vectorizer.get_feature_names())
vectorized_df.index = data.index
vectorized_df

,aaaaah,aaaaahhhhhhh,aaaaauuugghhhhhh,aaaahhhhh,aaah,aah,abc,ability,abject,able,...,zealand,zee,zen,zeppelin,zero,zillion,zombie,zone,zoo,éclair
Lousic C.K.,0,0,0,0,0,3,0,0,0,1,...,0,0,0,0,2,0,0,0,0,0
Dave Chappelle,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ricky Gervais,0,0,0,0,0,0,0,1,1,2,...,0,0,0,0,0,0,0,0,1,0
Bo Burham,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
Bill Burr,1,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,1,2,1,0,0
Jim Jefferies,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
John Mulaney,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,1
Hasan Minhaj,0,0,0,0,0,0,0,0,0,1,...,0,2,1,0,1,0,0,0,0,0
Ali Wong,0,0,0,0,0,0,1,0,0,2,...,0,0,0,0,0,0,1,0,0,0
Anthony Jeselnik,0,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0


## Save vectorized data

In [42]:
vectorized_df.to_csv('saves/vectorized_transcripts_df.csv')

## Analysis

### Most Common Words